<a href="https://colab.research.google.com/github/masayasato0407/scikit_learn_RF/blob/main/sklearn_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# scikit-optimizeのインストール
!pip install scikit-optimize

In [ ]:
# 必要なライブラリのインポート
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.inspection import permutation_importance

In [ ]:
# training dataのアップロード
uploaded = files.upload()

In [ ]:
# test dataのアップロード
uploaded = files.upload()

In [ ]:
# CSVファイルの読み込み
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
# 説明変数と目的変数 (event)の分割
X_train = train_data.drop('event', axis=1)
y_train = train_data['event']
X_test = test_data.drop('event', axis=1)
y_test = test_data['event']

In [ ]:
# ハイパーパラメータの範囲を定義
param_space = {
    'n_estimators': (10, 250),
    'max_depth': (1, 50),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999)
}

In [ ]:
# ベイズ最適化を用いたハイパーパラメータチューニング (時間がかかるためn_iterは10にしていますが、実際は50や100などにして下さい)
#n_iter: ハイパーパラメータ探索で試す組み合わせの回数を指定します。例えば、n_iter=32なら、32通りのハイパーパラメータを評価します。
#cv: トレーニングデータをいくつに分けるかの指定 (3ならデータを3つに分けて各部分を一度ずつvalidation setに使用)
opt = BayesSearchCV(
    estimator=RandomForestClassifier(random_state=1),
    search_spaces=param_space,
    n_iter=10,
    cv=3,
    n_jobs=-1,
    random_state=1
)

In [ ]:
# モデルのトレーニング
opt.fit(X_train, y_train)

In [ ]:
# 最適なハイパーパラメータでのモデル評価
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

In [ ]:
# 正診率とAUCの計算
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'Accuracy: {accuracy}')
print(f'ROC AUC: {roc_auc}')

In [ ]:
# Permutation Importanceによる変数重要度の計算
result = permutation_importance(best_model, X_test, y_test, n_repeats=30, random_state=1)
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': result.importances_mean})
importance_df.sort_values(by='Importance', ascending=False, inplace=True)
print(importance_df)

In [ ]:
# 予測結果をCSVに出力
output_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Probability': y_pred_proba})
output_df.to_csv('prediction_results.csv', index=False)

In [ ]:
# CSVファイルをダウンロードする
files.download('prediction_results.csv')